In [1]:
from imgaug import augmenters as iaa
from keras import applications, optimizers
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
import keras
import os, csv
import shutil
import imgaug as ia
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# dimensions the images.
img_width, img_height = 512, 384

train_data_dir = './dataset/train'
test_data_dir = './dataset/test'
validation_data_dir = './dataset/validation'
original_data_dir = './dataset-resized'

batch_size = 16
epochs = 20

In [3]:
def buildModel():
    # build the VGG16 network
    base_model = applications.VGG16(weights='imagenet', 
                                    include_top=False,
                                    input_tensor=Input(shape=(img_width, img_height, 3)))

    for layer in base_model.layers:
        layer.trainable = False
        
    top_model = base_model.output
    top_model = Flatten(name="Flatten")(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(256, activation='relu')(top_model)
    top_model = Dense(6, activation='softmax')(top_model)
    
    model = Model(inputs=base_model.input, outputs=top_model)

    model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

In [4]:
def createStratifiedData():
    # create dataframe with the images filenames
    dataset_files = open('./dataset.csv', 'w+')
    writer = csv.writer(dataset_files)

    writer.writerow(['image','class'])
    for path, dirs, files in os.walk(original_data_dir):
        for filename in files:
            if( filename != '.DS_Store'):
                writer.writerow([filename, os.path.basename(path)])

    # dataframe containing the filenames of the images and the classes
    df = pd.read_csv('./dataset.csv')
    df_y = df['class']
    df_x = df['image']

    skf = StratifiedShuffleSplit(n_splits = 1, test_size=0.2)

    for train_index, test_index in skf.split(df_x, df_y):
        x_train, x_test = df_x[train_index], df_x[test_index]
        y_train, y_test = df_y[train_index], df_y[test_index]

        train = pd.concat([x_train, y_train], axis=1)
        test = pd.concat([x_test, y_test], axis = 1)
        # take 20% of the training data from this fold for validation during training
        validation = test.sample(frac = 0.5)

        # make sure validation data does not include training data
        train = train[~train['image'].isin(list(validation['image']))]

        # copy the images according to the fold
        copy_images(train, 'train')
        copy_images(validation, 'validation')
        copy_images(test, 'test')

In [5]:
# used to copy files according to each fold
def copy_images(dataframe, directory):
    destination_directory = './dataset/{}'.format(directory)
    print('copying {} files to {}...'.format(directory, destination_directory))

    # remove all files from previous fold
    if os.path.exists(destination_directory):
        shutil.rmtree(destination_directory)

    # create folder for files from this fold
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # create subfolders for each class
    for image_class in set(list(dataframe['class'])):
        if not os.path.exists(destination_directory + '/' + image_class):
            os.makedirs(destination_directory + '/' + image_class)

    # copy files for this fold from a directory holding all the files
    for i, row in dataframe.iterrows():
        try:
            # this is the path to all of your images kept together in a separate folder
            path_from = '{}/{}/{}'.format(original_data_dir, row['class'],row['image'])
            path_to = "{}/{}".format(destination_directory, row['class'])
            
            # move from folder keeping all files to training, test, or validation folder (the "directory" argument)
            shutil.copy(path_from, path_to)
        except Exception as e:
            print("Error when copying {}: {}".format(row['image'], str(e)))

In [38]:
def augmentData():        
    # create numpy array with images paths
    img_paths = np.array([])
    for path, dirs, files in os.walk(train_data_dir):
        for filename in files:
            if(filename != '.DS_Store'):
                path_name = '{}/{}/{}'.format(train_data_dir,os.path.basename(path),filename)
                img_paths = np.append(img_paths, path_name)   
                
    # convert images to numpy arrays
    paths_number, = img_paths.shape
    
    images = np.zeros(shape=(paths_number,img_height,img_width,3),dtype='uint8')
    for idx, img_path in enumerate(img_paths):
        print('Creating array from ', img_path)
        img = cv2.imread(img_path, 1)
        im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images[idx] = im_rgb
        
    # aug configurations
    seq = iaa.Sequential([
        iaa.Affine(scale=(0.8, 1.3), mode=['edge']), # Scale images to a value between 80% and 150%
        iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, mode=['edge']), # Translate images by -10 to +10% on x- and y-axis independently
        iaa.Affine(rotate=(-25, 25), mode=['edge']), # Rotate images by -45 to 45 degrees
        iaa.GaussianBlur(sigma=(0.0, 1.0)), # Blur each image with a gaussian kernel with a sigma of 2.0:
        iaa.AdditiveGaussianNoise(scale=(0, 0.01*255)), # Add gaussian noise to an image, sampled once per pixel from a normal distribution N(0, s), where s is sampled per image and varies between 0 and 0.05*255
        iaa.Fliplr(0.5), # Flip 50% of all images horizontally
        iaa.Flipud(0.5), # Flip 50% of all images vertically
        iaa.CropAndPad(percent=(-0.1, 0.1), pad_mode=['edge']) # Crop or pad each side by up to 10 percent relative to its original size
    ], random_order=True)

    # create augmented images
    images_aug = seq.augment_images(images)
    
    # save augmented images
    for i in range(len(img_paths)):
        img_dest = '{}_AUG.jpg'.format(img_paths[i][:-4])  
        Image.fromarray(images_aug[i]).save(img_dest)
        print('Saving ',img_dest)

In [ ]:
def generateData(batch_size):
    train_datagen = ImageDataGenerator()

    validation_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=True,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    
    return [train_generator,validation_generator]

In [ ]:
def fineTuneModel(model, train_generator, validation_generator, epochs, batch_size):
    training_samples = 0
    validation_samples = 0
    
    for path, dirs, files in os.walk(train_data_dir):
        for filename in files:
            training_data += 1 

    for path, dirs, files in os.walk(validation_data_dir):
        for filename in files:
            validation_data += 1 
        
    model.fit_generator(
        train_generator,
        steps_per_epoch=training_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_samples // batch_size)
    
    return model

In [ ]:
model = buildModel()
train_generator,test_generator = generateData(batch_size)
trained_model = fineTuneModel(model,train_generator, test_generator, epochs, batch_size)
metrics = getMetrics(trained_model, batch_size)

In [7]:
createStratifiedData()

copying train files to ./dataset/train...
copying validation files to ./dataset/validation...
copying test files to ./dataset/test...


In [39]:
augmentData()

Creating array from  ./dataset/train/paper/paper5.jpg
Creating array from  ./dataset/train/paper/paper2.jpg
Creating array from  ./dataset/train/paper/paper3.jpg
Creating array from  ./dataset/train/paper/paper1.jpg
Creating array from  ./dataset/train/metal/metal2.jpg
Creating array from  ./dataset/train/metal/metal3.jpg
Creating array from  ./dataset/train/metal/metal1.jpg
Creating array from  ./dataset/train/metal/metal4.jpg
Creating array from  ./dataset/train/cardboard/cardboard4.jpg
Creating array from  ./dataset/train/cardboard/cardboard6.jpg
Creating array from  ./dataset/train/cardboard/cardboard3.jpg
Creating array from  ./dataset/train/cardboard/cardboard2.jpg
Creating array from  ./dataset/train/trash/trash4.jpg
Creating array from  ./dataset/train/trash/trash2.jpg
Creating array from  ./dataset/train/trash/trash3.jpg
Creating array from  ./dataset/train/trash/trash1.jpg
Creating array from  ./dataset/train/glass/glass3.jpg
Creating array from  ./dataset/train/glass/glass2.